In [17]:
import os
import PyPDF2
import openai
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from dotenv import load_dotenv
import time
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [18]:
load_dotenv()
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
llm = ChatOllama(model="llama3.2")

In [3]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        return " ".join(page.extract_text() or "" for page in reader.pages)


def extract_sections(resume_text):
    sections = {"projects": "", "experience": ""}
    current = None
    for line in resume_text.splitlines():
        l = line.lower().strip()
        if "experience" in l:
            current = "experience"
        elif "project" in l:
            current = "projects"
        elif any(x in l for x in ["education", "skills", "summary", "technical", "certification"]):
            current = None
        if current:
            sections[current] += line.strip() + " "
    return sections

In [4]:
def compute_bert_score(resume_text, jd_text):
    emb1 = bert_model.encode(resume_text, convert_to_tensor=True)
    emb2 = bert_model.encode(jd_text, convert_to_tensor=True)
    return round(util.pytorch_cos_sim(emb1, emb2).item() * 100, 2)


In [13]:
import re

def compute_llama_scores_single_call(jd_text, experience_text, project_text):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful resume evaluator."),
        ("user", """
You are a resume evaluator.

Given the job description and the candidate's resume sections below, score the candidate on a scale of 0–100 for each section separately:
1. Experience
2. Projects

Return only the two scores in JSON format as:
{{"experience_score": 88, "project_score": 76}}

Job Description:
\"\"\"{jd_text}\"\"\"

Experience Section:
\"\"\"{experience_text}\"\"\"

Project Section:
\"\"\"{project_text}\"\"\"
""")
    ])

    try:
        chain = prompt | llm
        response = chain.invoke({
            "jd_text": jd_text,
            "experience_text": experience_text[:1500],
            "project_text": project_text[:1500]
        })
        reply = response.content.strip()
        print("LLaMA raw response:", reply)

        try:
            scores = json.loads(reply)
            exp_score = float(scores.get("experience_score", 0))
            proj_score = float(scores.get("project_score", 0))
        except:
            exp_match = re.search(r'experience[^:]*[:\-]?\s*(\d{1,3})', reply, re.I)
            proj_match = re.search(r'project[^:]*[:\-]?\s*(\d{1,3})', reply, re.I)
            exp_score = float(exp_match.group(1)) if exp_match else 0
            proj_score = float(proj_match.group(1)) if proj_match else 0

        return exp_score, proj_score

    except Exception as e:
        print("Error scoring with LLaMA:", e)
        return 0, 0

def combine_scores(bert_score, llama_avg_score, w_bert=0.6, w_llama=0.4):
    return round(w_bert * bert_score + w_llama * llama_avg_score, 2)



In [14]:
resume_dir = "resumes/"
jd_path = "JDs/Data_engineering_intern_LiveRamp.txt"

with open(jd_path, 'r', encoding='utf-8') as f:
    jd_text = f.read()

results = []



In [15]:
for file in os.listdir(resume_dir):
    if not file.endswith(".pdf"):
        continue
    resume_path = os.path.join(resume_dir, file)
    resume_text = extract_text_from_pdf(resume_path)
    bert_score = compute_bert_score(resume_text, jd_text)
    sections = extract_sections(resume_text)
    experience_text = sections.get("experience", "")
    project_text = sections.get("projects", "")
    exp_score, proj_score = compute_llama_scores_single_call(jd_text, experience_text, project_text)
    llama_avg = (exp_score + proj_score) / 2
    final = combine_scores(bert_score, llama_avg)
    results.append({
        "Resume": file,
        "BERT Score": bert_score,
        "llama_score": llama_avg,
        "Final Score": final
    })

results_df = pd.DataFrame(results).sort_values("Final Score", ascending=False).reset_index(drop=True)

LLaMA raw response: Based on the job description and candidate's resume sections, I evaluate the candidate as follows:

* Experience Score: 92
	+ The intern has relevant experience in data quality monitoring, ETL pipelines, and serverless data retrieval systems.
	+ They have led cross-functional initiatives and implemented solutions that improve data accuracy, processing time, and performance.
	+ However, the internship duration is relatively short (6 months), which might be a concern for some employers.

* Project Score: 84
	+ The intern has demonstrated expertise in streamlining graph generation using R.
	+ They have also performed hypothesis testing and analyzed distribution similarities across datasets using T-tests and Q-Q plots.
	+ However, the project descriptions are brief, and more details about the context and challenges would strengthen the evaluation.

Note that these scores are subjective and based on my interpretation of the job description and candidate's resume sections

In [16]:
results_df

,Resume,BERT Score,llama_score,Final Score
0,Dhruvraj_resume_USHunger.pdf,53.87,91.0,68.72
1,Dhruvraj_Resume_intern_rocket.pdf,53.45,82.0,64.87
2,Dhruvraj_resume_May18.pdf,44.75,84.0,60.45
3,Dhruvraj_resume_Mar11.pdf,40.43,88.0,59.46
4,Dhruvraj_resume_MSDS.pdf,35.98,92.0,58.39
5,Dhruvraj_resume_image_analytics.pdf,25.18,82.0,47.91
